In [2]:
from data import OptiverDataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
import os

In [3]:
class OptiverNet(torch.nn.Module):
    def __init__(self, num_layers=10, hidden_size=256, dropout=0.3):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(torch.nn.Linear(14, hidden_size))
        for i in range(num_layers):
            self.layers.append(torch.nn.Linear(hidden_size, hidden_size))
            self.layers.append(torch.nn.ReLU())
            self.layers.append(torch.nn.Dropout(dropout))
        self.layers.append(torch.nn.Linear(hidden_size, 1))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [4]:
def collate_fn(data):
    return torch.stack([item[0] for item in data]), torch.stack([item[1] for item in data])

In [5]:
def train_lstm(hidden_size, layers, dropout, num_epochs=5): 
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = OptiverNet(hidden_size=hidden_size, layers=layers, dropout=dropout)
    model = model.to(device)
    
    batch_size = 1
    
    train_dataset = OptiverDataset(split='train')
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    val_dataset = OptiverDataset(split='val')
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    
    maes = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}')
        model.train()
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
                
        model.eval()
        loss = 0
        num_items = 0
        progress = tqdm(total=len(val_dataloader), desc='Validating')
        for x, y in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss += torch.sum(torch.absolute(y_pred - y)).item()
            num_items += torch.numel(y_pred)
            progress.update()
        progress.close()
        mae = loss/num_items
        maes.append(mae)
        print(f'MAE = {mae}')
    return min(maes)


In [6]:
import pandas as pd
hidden_sizes = [64, 128, 256]
layers = [2, 4, 6, 8]
dropouts = [0.0, 0.1, 0.2, 0.3, 0.4]

hidden_size_df = pd.DataFrame(columns=['hidden_size', 'mae'])
layer_df = pd.DataFrame(columns=['num_layers', 'mae'])
dropout_df = pd.DataFrame(columns=['dropout', 'mae'])

for hidden_size in hidden_sizes:
    mae = train_lstm(hidden_size=hidden_size, layers=4, dropout=0)
    hidden_size_df = hidden_size_df.append({'hidden_size': hidden_size, 'mae': mae}, ignore_index=True)

for num_layers in layers:
    mae = train_lstm(hidden_size=128, layers=num_layers, dropout=0)
    layer_df = layer_df.append({'num_layers': num_layers, 'mae': mae}, ignore_index=True)

for dropout in dropouts:
    mae = train_lstm(hidden_size=128, layers=4, dropout=dropout)
    dropout_df = dropout_df.append({'dropout': dropout, 'mae': mae}, ignore_index=True)

hidden_size_df
layer_df
dropout_df

Loading data: 100%|██████████| 20/20 [00:00<00:00, 230.86it/s]

Epoch 1


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 5.994354672520989
Epoch 2


...

MAE = 5.992745122965976
Epoch 5


Training:   0%|          | 0/139 [00:00<?, ?it/s]

Validating:   0%|          | 0/20 [00:00<?, ?it/s]

MAE = 5.975752631533131


,dropout,mae
0,0.0,5.969131
1,0.1,5.966392
2,0.2,5.978638
3,0.3,5.980938
4,0.4,5.975753


In [7]:
hidden_size_df

,hidden_size,mae
0,64.0,5.960039
1,128.0,5.955404
2,256.0,5.975110


In [9]:
layer_df

,num_layers,mae
0,2.0,5.963617
1,4.0,5.966758
2,6.0,5.982367
3,8.0,5.966689


In [5]:
def test():
    model = OptiverNet(hidden_size=64, num_layers=2, dropout=0.2)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    train_dataset = OptiverDataset(split='train_val')
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    loss_fn = nn.L1Loss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001)
    model.train()
    for i in range(5):
        progress = tqdm(total=len(train_dataloader), desc='Training')
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x).squeeze(dim=2)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progress.set_postfix({'loss': loss.item()})
            progress.update()
        progress.close()
        
    test_dataset = OptiverDataset(split='test')
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    model.eval()
    loss = 0
    num_items = 0
    progress = tqdm(total=len(test_dataloader), desc='Testing')
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x).squeeze(dim=2)
        loss += torch.sum(torch.absolute(y_pred - y)).item()
        num_items += torch.numel(y_pred)
        progress.update()
    progress.close()
    mae = loss/num_items
    return mae
mae = test()

Loading data: 100%|██████████| 159/159 [00:00<00:00, 273.98it/s]


Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Training:   0%|          | 0/159 [00:00<?, ?it/s]

Loading data: 100%|██████████| 40/40 [00:00<00:00, 224.56it/s]


Testing:   0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
mae

5.683152023472896
